In [2]:
import numpy as np
train_pkl = '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train.pkl'
train_np = np.load(train_pkl, allow_pickle=True)
split7_train_np = train_np[::7]
np.save('/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train_7split.pkl', split7_train_np)

In [13]:
import mmcv
train_pkl = '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train.pkl'
train_np = mmcv.load(train_pkl,)
split7_train_np = train_np[::7]
mmcv.dump(split7_train_np, '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train_7split.pkl')

/media/ddd/anaconda3/envs/depth/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 101: invalid device ordinal (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:115.)
  return torch._C._cuda_getDeviceCount() > 0


In [1]:
import mmcv
train_pkl = '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train.pkl'
train_np = mmcv.load(train_pkl,)
split7_train_np = train_np[::7]
mmcv.dump(split7_train_np, '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train_7split.pkl')
split7_train_np = train_np[15050:15150]
mmcv.dump(split7_train_np, '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train_100data.pkl')
split7_train_np = train_np[15050:15070]
mmcv.dump(split7_train_np, '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train_20data.pkl')


In [2]:
split7_train_np = train_np[15050:15060]
mmcv.dump(split7_train_np, '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_val_10data.pkl')
print(len(split7_train_np))
for i in range(9):
    split7_train_np += train_np[15050:15060]
print(len(split7_train_np))
mmcv.dump(split7_train_np, '/media/ddd/git/spa_AeDet/data/nuScenes/nuscenes_12hz_infos_train_100data.pkl')

10
100


In [56]:
import torch.nn.functional as F
import torch

#kernel_size
kernel_num = 7
#y의 위치
k = 3
#x의 위치
x = 10
stride = 1
h, w = 64,256 

indices = torch.arange(1, 65) 
off_mat = 32 / indices  

out_h, out_w = h // stride, w // stride 
kernel_div2 = int(kernel_num//2)

values = torch.arange(out_w)
M = values.view(out_w, 1, 1).repeat(1, kernel_num, kernel_num)
# print(values.view(out_w, 1, 1))
M = M.unsqueeze(0).repeat(64, 1, 1, 1)                         


kernel_offset = torch.arange(-int(kernel_num//2), kernel_num//2 + 1)
kernel_offset = kernel_offset.unsqueeze(0).repeat(kernel_num,1).unsqueeze(0).unsqueeze(0)
M = M + kernel_offset
print(M.shape)
print(M[0][:,5,3])
kernel = torch.arange(-int(kernel_num//2), kernel_num//2 + 1)
cnn_matrix = torch.zeros(out_w, kernel_num)
cnn_matrix = (off_mat.view(-1, 1) * kernel).float()

#print(cnn_matrix.shape)

first_row = cnn_matrix[0].unsqueeze(0).repeat(int(kernel_num//2), 1)
input_tensor = torch.cat((first_row, cnn_matrix), dim=0)
last_row = cnn_matrix[-1].unsqueeze(0).repeat(int(kernel_num//2), 1)
input_tensor = torch.cat((input_tensor, last_row), dim=0)

cnn_matrix = torch.zeros(out_w, kernel_num)

input_tensor_uq = input_tensor.unsqueeze(dim = 0).unsqueeze(dim = 1)
patches = F.unfold(input_tensor_uq, kernel_num, stride=stride)
# Reshape to get the patches in the desired format
patches = patches.transpose(1, 2).reshape(-1, 1, kernel_num, kernel_num)
x_offset = patches.repeat(1,out_w,1,1)
y_offset = torch.zeros(x_offset.shape)

changed_x_offset = x_offset + M

x_offset[changed_x_offset < 0] = changed_x_offset[changed_x_offset < 0] + 128 
x_offset[changed_x_offset > 128] = changed_x_offset[changed_x_offset > 128] - 256 
conv_offset = torch.stack([y_offset, x_offset])
print(conv_offset.shape)

#print(y_offset)
#[2, 64, 128, 7, 7] -> [1,98,64,128]
conv_offset = torch.stack([x_offset, y_offset]).permute(3,4,0,2,1).contiguous().view(1,2*kernel_num*kernel_num,out_w,out_h) 

torch.Size([64, 256, 7, 7])
tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173

In [43]:
import torch

stride = [2, 2]
size = [256, 64]


h, w = size
out_h, out_w = h // stride[0], w // stride[1]
out_num = out_h * out_w

kh, kw = 7,7
kernel_num = kh * kw
grid_x = torch.arange(-((kw - 1) // 2), kw // 2 + 1)
grid_y = torch.arange(-((kh - 1) // 2), kh // 2 + 1)
grid_x = grid_x.view(1, kw).repeat(kh, 1)
grid_y = grid_y.view(kh, 1).repeat(1, kw)
conv_offset = torch.stack([grid_y, grid_x]).permute(1, 2, 0).contiguous().view(-1)
conv_offset = conv_offset.view(1, kernel_num, 2).repeat(out_num, 1, 1).type(torch.atan2(torch.tensor(1), torch.tensor(1)).type())
conv_offset_orig = torch.stack([grid_y, grid_x]).permute(1, 2, 0).contiguous().view(-1)
conv_offset_orig = conv_offset_orig.view(1, kernel_num, 2).repeat(out_num, 1, 1).type(torch.atan2(torch.tensor(1), torch.tensor(1)).type())

# compute the offset of PolarConv
h_index = (torch.arange(out_h) + 1) * stride[0]
h_index = h_index - h_index[0]
w_index = (torch.arange(out_w) + 1) * stride[1]
w_index = w_index - w_index[0] + 1

index_repeated = w_index.unsqueeze(0).repeat(out_h,1).view(-1, 1)
conv_index = index_repeated + conv_offset[:,:,1]
conv_index[conv_index<=0] = (w//2) / max(kw//2, 1)
offset = w//2 / conv_index # / (max(kw//2, 1))
conv_offset[:,:,0] *= offset

# make out index to inner
h_index_temp = h_index[:, None, None, None, None].repeat(1, out_w, kh, kw, 2).view(*conv_offset.shape)

conv_offset_temp2 = conv_offset + h_index_temp
conv_offset -= conv_offset_orig
conv_offset[conv_offset_temp2 < 0] += h
conv_offset[conv_offset_temp2 > h-1] -= h

# if stride is bigger than 1, the center of kernel is not on the integer index.s
# shift_h = (h - 7) % stride[0]
# shift_w = (w - 7) % stride[1]
# conv_offset[:, :, 0] += shift_w / 2.0
# conv_offset[:, :, 1] += shift_h / 2.0

conv_offset = conv_offset.contiguous().view(1, out_h, out_w, 2 * kernel_num).permute(0, 3, 1, 2)

In [44]:
conv_offset.shape

torch.Size([1, 98, 128, 32])

In [46]:
conv_offset[0,:,100,0].reshape(7,7,2)[:,:,0]

tensor([[  -6.0000,   -6.0000,   -6.0000,  -93.0000,  -45.0000,  -29.0000,
          -21.0000],
        [  -4.0000,   -4.0000,   -4.0000,  -62.0000,  -30.0000,  -19.3333,
          -14.0000],
        [  -2.0000,   -2.0000,   -2.0000,  -31.0000,  -15.0000,   -9.6667,
           -7.0000],
        [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000,    0.0000,
            0.0000],
        [   2.0000,    2.0000,    2.0000,   31.0000,   15.0000,    9.6667,
            7.0000],
        [   4.0000,    4.0000,    4.0000, -194.0000,   30.0000,   19.3333,
           14.0000],
        [   6.0000,    6.0000,    6.0000, -163.0000,   45.0000,   29.0000,
           21.0000]])

In [48]:
import json
with open('/media/ddd/git/spa_AeDet/outputs/overfit/metrics_summary.json') as file:
    # Load the JSON data
    data = json.load(file)
print("mAP:", round(data['mean_ap'], 4))
print("mATE:", round(data['tp_errors']['trans_err'], 4))
print("mASE:", round(data['tp_errors']['scale_err'], 4))
print("mAOE:", round(data['tp_errors']['orient_err'], 4))
print("mAVE:", round(data['tp_errors']['vel_err'], 4))
print("mAOE:", round(data['tp_errors']['attr_err'], 4))
print("NDS:", round(data['nd_score'], 4))

mAP: 0.4808
mATE: 0.6275
mASE: 0.4517
mAOE: 0.3778
mAVE: 0.4269
mAOE: 0.3752
NDS: 0.5145


In [59]:
import torch
import torch.nn.functional as F

# Input feature map
# This could represent a 1x1 image with 1 channel (for simplicity)
input = torch.ones(1, 1, 16, 44)

# Flow field grid
# In this case, it's a 1x1 grid with an offset of 0.2 in the x and y directions
# The values should be in the range of [-1, 1], where -1 represents the left/top boundary, 1 represents the right/bottom boundary, and 0 is the center.
grid = torch.Tensor([[[[0,0]]]])

for i in range(16):
    for j in range(44):
        input[:,:,i,j] = 100*i + j
# Resample the input based on the grid
output = F.grid_sample(input, grid, mode='bilinear', padding_mode='zeros')

print(input[:,:,7,21])
print(grid)
print(output)

tensor([[721.]])
tensor([[[[0., 0.]]]])
tensor([[[[771.5000]]]])


In [60]:
import torch.nn as nn

# Create a Conv2d layer
conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride=1, padding=1, bias=True)

# Print the initial bias values
print(conv.bias.data)

tensor([0.1243])


In [1]:
from PIL import Image

# Open the image file
image = Image.open('/media/ddd/shift/discrete/images/val/front/img/6c37-2ac8/00000030_img_front.jpg')

# Display some information about the image
print('Image format:', image.format)
print('Image size:', image.size)

# You can perform various operations on the image here
# For example, you can save it in a different format or display it

# Close the image file
image.close()


Image format: JPEG
Image size: (1280, 800)


In [ ]:
import smtplib
from email.mime.text import MIMEText
 
smtp = smtplib.SMTP('spa.hanyang.ac.kr', 587)
smtp.ehlo()      # say Hello
smtp.starttls()  # TLS 사용시 필요
smtp.login('junghokim@spa.hanyang.ac.kr', 'kjh9653102')
 
msg = MIMEText('본문 테스트 메시지')
msg['Subject'] = '테스트'
msg['To'] = 'junghokim@spa.hanyang.ac.kr'
smtp.sendmail('junghokim@spa.hanyang.ac.kr', 'junghokim@spa.hanyang.ac.kr', msg.as_string())
 
smtp.quit()